In [ ]:
# Initialize OK
from client.api.notebook import Notebook
ok = Notebook('homework1.ok')

# Homework 1
## PSTAT 134/234 (Spring 2019)

![election_prediction](http://donsnotes.com/politics/images/clinton-trump-rating.png)

We haven't yet talked about predictive models in class yet, but we can at least think about what makes a "good" prediction.  In this assignment, we'll focus on evaluating the quality of election predictions made by the website [fivethirtyeight.com](https://www.fivethirtyeight.com).  As one prominent example, fivethirtyeight predicted that Clinton had a 70.9% chance to win the election.  Was their model wrong?

To gain insight into questions like this, we'll focus on  [US House elections predictions from 2018](https://projects.fivethirtyeight.com/2018-midterm-election-forecast/house/).  Their predictions are based predominantly on polling data but include other sources as well (state of the economy, overall favoriability of politic parties, etc).  

This homework is based loosely on [this article](https://projects.fivethirtyeight.com/checking-our-work/).  Please read the article before beginning the assignment.


In [1]:
!wget -nc https://raw.githubusercontent.com/fivethirtyeight/checking-our-work-data/master/us_house_elections.csv

`ls` command can be used to verify that the file is now listed in our folder.   

In [2]:
!ls -F -l

In [3]:
import pandas as pd ## call pandas functions using "pd"
import numpy as np  ## call numpy functions using "np"
election_data = pd.read_csv("us_house_elections.csv")

## Examining the Data

Display the first 10 rows of the dataset using `iloc`.

Fivethirtyeight has three different prediction models: `lite`, `classic` and `deluxe`, which roughly incorporate an increasing number of assumptions.  In this assignment lets focus on evaluting the quality of the `classic` predictions.  You can read more about how the prediction models work [here](https://fivethirtyeight.com/methodology/how-fivethirtyeights-house-and-senate-models-work/).

Fivethirtyeight continuously updated their predictions as more polling data became available for each of the races.  Let's focus on the predictions a few months before the election, on August 11th, and on the morning of election day, November 6th.  

### Question 1

<!--
BEGIN QUESTION
name: q1
manual: false
points: 2
-->

Create a new pandas dataframe called `election_sub` by filtering to include only rows in which the `forecast_type` is "classic", and the date of the foreceast (`forecast_date`) is 8/11 or 11/6.

In [5]:
election_sub = ...

In [ ]:
ok.grade("q1");

### Question 2

We want to check whether events predicted by 538 to occur with probability _close to_ X% actually occured about X% of the time.  To do this, we have to define _close_.  First, we'll define the `cut_points` as 20 equally spaced numbers between 0 and 1 using `np.linspace`.  Then we'll group the predicted probabilities into the `19` equally spaced bins determined by those cut points.  Define the bin for each observation using the `pd.cut` function on the `probwin` variable.  We'll assign the result to a new column of `election_sub` called `bin`.

<!--
BEGIN QUESTION
name: q2
manual: false
points: 2
-->

In [8]:
cut_points = np.linspace(0, 1, 20)
...

In [ ]:
ok.grade("q2");

### Question 3

<!--
BEGIN QUESTION
name: q3
manual: false
points: 2
-->

Now we've grouped the observations into a discrete set of bins according to the predicted probability, `probwin`.  Within each bin, we now want to compute the actual fraction of times the candidates won. If 538 did a good job, it will be close to the predited probabilities.   You'll need to use the `groupby` function to compute the mean of `probwin_outcome` (1 is a win and 0 is a loss) within each bin.  Save the fraction of actual wins in each bin in a list called `fraction_outcome`.

In [10]:
fraction_outcome = ...

In [ ]:
ok.grade("q3");

### Question 4

<!--
BEGIN QUESTION
name: q4a
manual: false
points: 2
-->

For this problem we'll make a plot of the predicted probabilities and actual fraction of wins in each bin.  We've already computed the actual fraction of wins; all that remains is to plot it against the predicted value assocaited with each bin.  For the predicted value in each bin, we'll use the midpoint of the bin.  Compute the midpoints of each bin from `cut_points`.

In [13]:
midpoints = ...
list(midpoints)

In [ ]:
ok.grade("q4a");

Now make a scatterplot using `midpoints` as the x variable and `fraction_outcome` as the y variable.  Draw a dashed line from `[0,0]` to `[1,1]` to mark the line y=x.  

<!--
BEGIN QUESTION
name: q4b
manual: true 
points: 2
-->
<!-- EXPORT TO PDF -->

In [16]:
%matplotlib inline
import matplotlib.pyplot as plt

...

### Question 5: adding error bars

If you did things correctly, it should look like fivethirtyeight has done "pretty" well with their forecasts: the actual fraction of wins tracks closely with the predicted number.  But how do we decide what's "good enough"? Consider this example: I correctly predict that a coin is fair (e.g. that it has a 50% chance of heads, 50% chance of tails).  But if I flip it 100 times, I can be pretty sure it won't come up heads exactly 50 times.  The fact that it didn't come up heads exactly 50 times doesn't make my prediction incorrect.

To assess how reasonable the predictions are, I need to quantify the uncertainty in my estimate.  It's reasonable to assume that within each bin, $k$, the observed number of wins, $Y_k \sim Bin(n_k, p_k)$, where $n_k$ is the number of elections and $p_k$ is the predicted win probability in bin $k$.

Classical results tell us that the obseved fraction of wins in bin $k$, $\hat p = \frac{Y_k}{n_k}$ has variance Var$\left(\hat p_k\right) = \frac{p_k (1-p_k)}{n_k} \approx \frac{\hat p_k(1- \hat p_k)}{n_k}$.  The standard deviation of the Binomial proportion then is $\hat \sigma_k \approx \sqrt{\frac{\hat p_k(1- \hat p_k)}{n_k}}$.  

If we use the [normal approximation to generate a confidence interval](https://en.wikipedia.org/wiki/Binomial_proportion_confidence_interval#Normal_approximation_interval), then the 95% interval has the form $\hat p_k \pm 1.96 \hat \sigma_k$.

Create a new "aggregated" dataframe.  This time, group `election_sub` by the `bin` and compute both the average of the `probwin_outcome` (`mean`) and the number of observations in each bin (`count`) using the `agg` function.  Call this new data frame, `election_agg`.

<!--
BEGIN QUESTION
name: q5a
manual: true
points: 2
-->
<!-- EXPORT TO PDF -->

In [17]:
election_agg = ...

Use the `mean` and `count` columns of `election_agg` to create a new column of `election_agg` titled `err`, which stores $1.96 \times \hat \sigma_k$ in each bin $k$.

<!--
BEGIN QUESTION
name: q5b
manual: true
points: 2
-->
<!-- EXPORT TO PDF -->

In [18]:
...

Use `plt.errorbar` to create a new plot with error bars associated with the actual fraction of wins in each bin.  Again add a dashed y=x line.  Set the argument `fmt='.'` to create a scatterplot with errorbars.

In [19]:
plt.errorbar(midpoints, election_agg['mean'].values, yerr=election_agg['err'].values, fmt='.')
plt.plot([0, 1], [0, 1], '--')

### Question 6: computing the coverage

If our intervals were true 95% confidence intervals, then we would expect about 95% of them to cover the midpoint of the bin (i.e. overlap with the y=x line).  What fraction of the 95% confidence intervals cover the bin midpoint?  Create a list called `upper` to be the `mean` + `err` and another `lower` to be `mean` - `err`.  Next, compute `frac_covering` as the fraction of midpoints between `lower` and `upper`. 

<!--
BEGIN QUESTION
name: q6
manual: false
points: 2
-->

In [20]:
upper = ...
lower = ...

frac_covering = ...

In [ ]:
ok.grade("q6");

### Question 7: understanding confidence intervals

<!--
BEGIN QUESTION
name: q7
manual: true
points: 2
-->

Are the 95% confidence intervals generally larger or smaller for more confident predictions (e.g. the predictions closer to 0 or 1).  What are the factors that determine the length of the confidence intervals?
<!-- EXPORT TO PDF -->

**SOLUTION HERE**

### Question 8: finding the candidate that had the biggest change in support

Let's see if we can find the candidate that seemed to improve their standing the most between  August 11 and November 6.  First, fill in the function `abs_diff`, which takes in a pandas data frame and computes the difference between the largest values of `probwin` and the smallest value.  

<!--
BEGIN QUESTION
name: q8a
manual: true
points: 2
-->
<!-- EXPORT TO PDF -->

In [25]:
# Input: a pandas dataframe with a numeric column named `probwin`
# Output: a pandas dataframe with the same columns, with an additional column named `absdiff`
def abs_diff(x):
    ...
    return x

We can use this function to compute the difference between the maximum and minimum predicted with probabilities for every candidate.  To do so, group `election_sub` by `candidate` and `apply` the function `abs_diff`.  Find the index of the largest difference in `diff_dataframe` and store it in `max_idx`.  Do this using `np.nanargmax` function.  This function finds the _index_ of the largest value, ignoring any missing values (`nans`). 

<!--
BEGIN QUESTION
name: q8b
manual: true
points: 2
-->
<!-- EXPORT TO PDF -->

In [26]:
diff_dataframe = ...
max_idx = ...

<!--
BEGIN QUESTION
name: q8c
manual: false
points: 2
-->

In [27]:
candidate = ...

In [ ]:
ok.grade("q8c");

Did the candidate win or lose the election?
<!--
BEGIN QUESTION
name: q8d
manual: true
points: 2
-->
<!-- EXPORT TO PDF -->

**SOLUTION HERE**

### Question 9: plot predictions over time

Plot the forecasted win probability for the candidate you found above, for every available date.  For this you'll need to return to working with the full `election_data`, not `election_sub`.  Don't forget, you should still filter to `classic` forecasts only.  First, create an array of `predicted_probs` for the candidate at every date.  Also save the date in an array called `forecast_date`.  When creating `forecast_date`, use `pd.to_datetime` to convert a `str` datatype to the date format that can easily be plotted.

<!--
BEGIN QUESTION
name: q9a
manual: false
points: 2
-->

In [30]:
predicted_probs = ...
forecast_date = ...

In [ ]:
ok.grade("q9a");

Now create a lineplot with forecast date on the x-axis and the predicted win probability on the y-axis.
<!--
BEGIN QUESTION
name: q9b
manual: true
points: 2
-->
<!-- EXPORT TO PDF -->

In [34]:
...

### Question 10: prediction histograms

<!--
BEGIN QUESTION
name: q10a
manual: true
points: 2
-->

Make a histogram showing the predicted win probabilities on the morning of the election.  Again, restrict yourself to only the `classic` predictions.
<!-- EXPORT TO PDF -->

In [35]:
...

<!--
BEGIN QUESTION
name: q10b
manual: true
points: 2
-->

Are most house elections easy to forecast or hard to forecast?
<!-- EXPORT TO PDF -->

**SOLUTION HERE**

### Question 11: Comparing election and baseball predictions

Fivethirtyeight also builds predictive models for sporting events.  The following code will down a csv file containg their predictions for who would win every major league baseball game over the past two years.

In [36]:
!wget https://raw.githubusercontent.com/fivethirtyeight/checking-our-work-data/master/mlb_games.csv    

<!--
BEGIN QUESTION
name: q11a
manual: true
points: 2
-->

Create a pandas dataframe from the csv and print the first 10 rows.
<!-- EXPORT TO PDF -->

In [37]:
...

In this dataframe `prob1` is the predicted win probability for `team1`.  Make a histogram of `prob1`.  Set the limits of the x-axis to `[0, 1]`
<!--
BEGIN QUESTION
name: q11b
manual: true
points: 2
-->
<!-- EXPORT TO PDF -->

In [38]:
...

### Question 12

<!--
BEGIN QUESTION
name: q12
manual: true
points: 2
-->

Find the most "surprising" baseball game outcome.  To do so, select all of the entries for which `prob1_outcome` is 1 (i.e. `team1` won the game), and then look for the index of the row containing the smallest value of `prob1`.  This will correspond to the game that was most suprising according to fivethirtyeights predictions. Find and print the row corresponding to this most surprising outcome.
<!-- EXPORT TO PDF -->

In [39]:
...

### Question 13

<!--
BEGIN QUESTION
name: q13
manual: true
points: 2
-->

Are the outcomes of baseball games generally easier or harder to predict than the outcomes of political elections?  In a few sentences, comment on why this might be the case.  What data is available for these predictions? What factors affect the outcomes of elections and baseball games? What makes an event like an election or a baseballgame "random"?
<!-- EXPORT TO PDF -->

**SOLUTION HERE**

### Question 14 (PSTAT 234)

**(Question 5 revisited)** In Question 5, error bars were calculated by assuming that election outcomes are Binomial random variables.

On the other hand, if we do not have a good distributional about the election outcomes, we can estimate the error bars from resampling the data.

Write a function named `bootstrap_data_means` that can take a data frame, say `data_in`, as input. Suppose `data_in` has `n` rows. Inside `bootstrap_data_means` function, you will

1. Select `n`-rows of `data_in` at random with replacement, creating a pseudo-dataset
1. For each bin, compute probabilities of success for each pseudo-dataset as you have done on Question 5
1. Return the computed probabilities

<!--
BEGIN QUESTION
name: q14a
manual: false
points: 2
-->

Then, run `bootstrap_data_means` function 100 times, and create a data frame `bootstrap_election_100_agg`:

In [40]:
def bootstrap_data_means(data_in):
    
    from numpy.random import choice
    
    n = len(data_in)
    
    # randomly sample row indices with replacement
    indx = ...
    # compute probabilities of success
    bootstrap_means = ...
    
    return(bootstrap_means)

bootstrap_election_100_agg = ...

In [ ]:
ok.grade("q14a");

By now, we have a distribution of success probabilities saved in `bootstrap_election_agg`. We can compute empirical error bars from 2.5% and 97.5% quantiles. Write function named `bootstrap_error_bars` that can be used to calculate the following columns:
1. `mean`: mean of probabilities of success
1. `err_low`: low point of the error bars
1. `err_high`: high point of the error bars

<!--
BEGIN QUESTION
name: q14b
manual: false
points: 2
-->
Funtion `bootstrap_error_bars` is to be called by using `bootstrap_election_100_agg.apply(bootstrap_error_bars, ...)`.

In [45]:
def bootstrap_error_bars(x):
    ...
    
bootstrap_election_agg = ...

In [ ]:
ok.grade("q14b");

Recall the figure from Question 5. In addition, draw a horizontal lines at 0 and 1.

In [48]:
plt.errorbar(midpoints, election_agg['mean'].values, yerr=election_agg['err'].values, fmt='.')
plt.plot([0, 1], [0, 1], '--')
plt.plot([0, 1], [1, 1], ':r')
plt.plot([0, 1], [0, 0], ':r')

<!--
BEGIN QUESTION
name: q14c
manual: true
points: 2
-->
Create an analogous plot for empirical error bars with `bootstrap_election_agg`. Also draw a horizontal lines at 0 and 1.
<!-- EXPORT TO PDF -->

**SOLUTION HERE**

In [49]:
# Create a similar figure using empirical error bars  
...

<!--
BEGIN QUESTION
name: q14d
manual: true
points: 2
-->
Compare the two error bar plots and explain.
<!-- EXPORT TO PDF -->

**SOLUTION HERE**

# Submit
Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output.
**Please save before submitting!**

<!-- EXPECT 16 EXPORTED QUESTIONS -->

In [ ]:
# Save your notebook first, then run this cell to submit.
import jassign.to_pdf
jassign.to_pdf.generate_pdf('homework1.ipynb', 'homework1.pdf')
ok.submit()